# Mediator

It is a mechanism for facilitating communication between the components. Naturally, the mediator itself needs to be accessible to every component taking part, which means it should either be a global static variable or, alternatively, just a reference that gets injected into every component.

## simple example

In [69]:
# componnt

class Person:
    def __init__(self, name):
        self.name = name
        self.log = ""
        self.room = None

    def receive(self, msg):
        self.log.join(msg)
        print(f"[{self.name} Session] {msg}")

    def post(self, msg):
        self.room.broadcast(f"[From {self.name}] {msg}")

    def private_msg(self, who, msg):
        self.room.message(self.name, who, f"[From {self.name}] {msg}")

In [70]:
# mediator

class ChatRoom:
    def __init__(self):
        self.people = []

    def join(self, person):
        person.room = self
        self.people.append(person)
        self.broadcast(f"{person.name} has joint the room.")
        
    def broadcast(self, msg):
        for p in self.people:
            p.receive(msg)

    def message(self, sender, receiver, msg):
        for p in self.people:
            if p.name == receiver.name:
                p.receive(msg)

In [72]:
chatroom = ChatRoom()
john = Person("John")
lisa = Person("Lisa")
chatroom.join(john)
chatroom.join(lisa)

john.post("hi everyone")

dave = Person("Dave")
chatroom.join(dave)
dave.post("hello")

lisa.private_msg(dave, "welcome back")

[John Session] John has joint the room.
[John Session] Lisa has joint the room.
[Lisa Session] Lisa has joint the room.
[John Session] [From John] hi everyone
[Lisa Session] [From John] hi everyone
[John Session] Dave has joint the room.
[Lisa Session] Dave has joint the room.
[Dave Session] Dave has joint the room.
[John Session] [From Dave] hello
[Lisa Session] [From Dave] hello
[Dave Session] [From Dave] hello
[Dave Session] [From Lisa] welcome back


## Mediator + Event

In [29]:
# event

class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

In [21]:
# mediator

class Game:
    def __init__(self):
        self.events = Event()

    def update_board(self, args):
        self.events(args)
        

In [27]:
# component

class PlayerScoreData:
    def __init__(self, name, scores):
        self.player_name = name
        self.scores = scores

class Player:
    def __init__(self, name, game):
        self.name = name
        self.scores = 0
        self.game = game

    def score(self):
        self.scores += 1
        psd = PlayerScoreData(self.name, self.scores)
        self.game.update_board(psd)

In [25]:
# component

class ScoreBoard:
    def __init__(self, game):
        self.game = game
        self.game.events.append(self.refresh)

    def refresh(self, args):
        if isinstance(args, PlayerScoreData):
            print(f"{args.player_name} score: {args.scores}")

In [31]:
game = Game()
player1 = Player("John", game)
player2 = Player("Sam", game)
board = ScoreBoard(game)
player1.score()
player1.score()
player2.score()

John score: 1
John score: 2
Sam score: 1
